In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import onnxruntime as rt
import onnx
from skl2onnx.common.data_types import FloatTensorType
from skl2onnx import to_onnx
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_classif
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from skl2onnx import convert_sklearn
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

In [ ]:
# Let's load the dataset
data = pd.read_csv('../censored_dataset.csv')

# Let's specify the features and the target - Use all the features from the examples dataset
# You should drop Ja and Nee
y = data['checked']
X = data.drop(['checked', 'Ja', 'Nee'], axis=1)
X = X.astype(np.float32)

# Let's split the dataset into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
def get_features_to_drop(keep_data):
    columns_to_drop = [
        col for col in data.columns if col not in keep_data and col not in ["Ja", "Nee", "checked"]
    ]
    return columns_to_drop

In [ ]:
linear_classifier = LogisticRegression(
    penalty='l2',        
    solver='lbfgs',
    max_iter=1000,
    random_state=42
)

In [ ]:
keep_cols = [
	# censored features
]

feature_selector = ColumnTransformer([
        ('drop_sensitive', 'drop', get_features_to_drop(keep_cols)),
    ], remainder=StandardScaler())


In [ ]:
selector = VarianceThreshold(0.1)

In [ ]:
# Create a pipeline object with our selector and classifier
# NOTE: You can create custom pipeline objects but they must be registered to onnx or it will not recognise them
# Because of this we recommend using the onnx known objects as defined in the documentation
pipeline = Pipeline(steps=[
    ('remove_sensitive_features', feature_selector),
    ('feature selection', selector),
    ('classification', linear_classifier)
])

In [ ]:
# Let's train a simple model
pipeline.fit(X_train, y_train)

# Let's evaluate the model
y_pred = pipeline.predict(X_test)
original_accuracy = accuracy_score(y_test, y_pred)
print('Accuracy of the original model: ', original_accuracy)

In [ ]:
# Let's convert the model to ONNX
onnx_model = convert_sklearn(
    pipeline, initial_types=[('X', FloatTensorType((None, X.shape[1])))],
    target_opset=12)

# Let's check the accuracy of the converted model
sess = rt.InferenceSession(onnx_model.SerializeToString())
y_pred_onnx =  sess.run(None, {'X': X_test.values.astype(np.float32)})

accuracy_onnx_model = accuracy_score(y_test, y_pred_onnx[0])
print('Accuracy of the ONNX model: ', accuracy_onnx_model)

In [ ]:
# Let's save the model
onnx.save(onnx_model, "../onnx_models/censored.onnx")

In [ ]:
# Let's load the model
new_session = rt.InferenceSession("../onnx_models/censored.onnx")

# Let's predict the target
y_pred_onnx2 =  new_session.run(None, {'X': X_test.values.astype(np.float32)})

accuracy_onnx_model = accuracy_score(y_test, y_pred_onnx2[0])
print('Accuracy of the ONNX model: ', accuracy_onnx_model)